# [--- Day 12: Hot Springs ---](https://adventofcode.com/2023/day/12)


## Setup

Use the `input` file if present, otherwise use the sample input.

In [111]:
import numpy as np
is_sample = False

"""
???.### 1,1,3 - 1 arrangement
.??..??...?##. 1,1,3 - 4 arrangements
?#?#?#?#?#?#?#? 1,3,1,6 - 1 arrangement
????.#...#... 4,1,1 - 1 arrangement
????.######..#####. 1,6,5 - 4 arrangements
?###???????? 3,2,1 - 10 arrangements
"""

try:
  input = open("input", "r").read().splitlines()
except FileNotFoundError:
  input = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""
  input = input.split("\n")
  is_sample = True

print(input)

['???.### 1,1,3', '.??..??...?##. 1,1,3', '?#?#?#?#?#?#?#? 1,3,1,6', '????.#...#... 4,1,1', '????.######..#####. 1,6,5', '?###???????? 3,2,1']


## Functions and Classes



In [112]:
def get_combos(curr, length, springs, res=None):
    if len(curr) > length:
        curr = curr[:length]

    if not springs:
        curr += ("." * (length - len(curr)))
        if res is not None:
            res.append(curr[:length])
        return
    
    sp = springs[0]
    rest = springs[1:]
    spring = f"{'#' * sp}."

    for p in range(0, length-len(curr)-sp+1):
        spaces = "." * p
        next = f"{curr}{spaces}{spring}"
        get_combos(next, length, rest, res)

get_combos("###.", 10, [], [])
get_combos("###.", 10, [5])
get_combos("###.", 10, [1, 1])
get_combos("", 15, [1, 3, 1, 6])
get_combos("#.###.#.", 15, [6])

In [113]:
import re

def get_possible(mask, springs):
    mask = f"{mask}"
    print(mask, springs)

    results = []
    get_combos("", len(mask), springs, results)
    print("results", results)

    found = 0
    for res in results:
        lfound = 1
        for comp in zip(res, mask):
            #print(comp)
            if comp[0] == "#" and comp[1] == ".":
                lfound = 0
            if comp[0] == "." and comp[1] == "#":
                lfound = 0
        if lfound:
            found += 1

    print("found", found)
    return found

## Part 1

In [114]:
res = 0

for line in input:
    mask, springs = line.split()
    sp_list = [int(c) for c in springs.split(",")]
    res += get_possible(mask, sp_list)

print(res)
assert(res == 21)

???.### [1, 1, 3]
results ['#.#.###']
found 1
.??..??...?##. [1, 1, 3]
results ['#.#.###.......', '#.#..###......', '#.#...###.....', '#.#....###....', '#.#.....###...', '#.#......###..', '#.#.......###.', '#.#........###', '#..#.###......', '#..#..###.....', '#..#...###....', '#..#....###...', '#..#.....###..', '#..#......###.', '#..#.......###', '#...#.###.....', '#...#..###....', '#...#...###...', '#...#....###..', '#...#.....###.', '#...#......###', '#....#.###....', '#....#..###...', '#....#...###..', '#....#....###.', '#....#.....###', '#.....#.###...', '#.....#..###..', '#.....#...###.', '#.....#....###', '#......#.###..', '#......#..###.', '#......#...###', '#.......#.###.', '#.......#..###', '#........#.###', '.#.#.###......', '.#.#..###.....', '.#.#...###....', '.#.#....###...', '.#.#.....###..', '.#.#......###.', '.#.#.......###', '.#..#.###.....', '.#..#..###....', '.#..#...###...', '.#..#....###..', '.#..#.....###.', '.#..#......###', '.#...#.###....', '.#...#..###...', '.

## Part 2

In [115]:
from functools import lru_cache

@lru_cache
def find_matches(mask, pos, springs, spos, spring_used):
    if spos == len(springs):
        if "#" in mask[pos:]:
            # Used up all springs, but need more
            return 0
        else:
            return 1

    if pos == len(mask):
        return 0
   
    sp = springs[spos]
    if len(mask) - pos < sp:
        # Uhoh, no room left
        return 0

    # Need to find next chunk that can hold sp
    if mask[pos] == ".":
        # No spring can be placed
        return find_matches(mask, pos + 1, springs, spos, False)
    
    if mask[pos] == "#":
        # Have no choice, must try to place sp here
        if spring_used:
            # A spring was just set, so can't do anything
            return 0
        if "." in mask[pos:(pos+sp)]:
            # Can't fit a spring here
            return 0
        return find_matches(mask, pos + sp, springs, spos + 1, True)

    if mask[pos] == "?":
        # Option to either place a spring or not
        matches = 0
        if not spring_used and "." not in mask[pos:(pos+sp)]:
            matches += find_matches(mask, pos + sp, springs, spos + 1, True)
        return matches + find_matches(mask, pos + 1, springs, spos, False)
    raise Exception()

find_matches("???#", 0, (1, 1), 0, False)

2

In [116]:
sum = 0
print(input)
for line in input:
    mask, springs = line.split()
    mask = mask + "?" + mask + "?" + mask + "?" + mask + "?" + mask
    sp_list = [int(c) for c in springs.split(",")]
    sp_list = sp_list * 5
    res = find_matches(mask, 0, tuple(sp_list), 0, False)
    sum += res
print(sum)

assert(sum == 4546215031609 or sum == 525152)


['???.### 1,1,3', '.??..??...?##. 1,1,3', '?#?#?#?#?#?#?#? 1,3,1,6', '????.#...#... 4,1,1', '????.######..#####. 1,6,5', '?###???????? 3,2,1']
525152
